In [1]:
from board_labeler import get_files_in_dir

filesDir = "labeledConstSize/"

paths = get_files_in_dir(filesDir)

In [2]:
import cv2
import re
import os
from sklearn.preprocessing import StandardScaler
from board_labeler import dictClasses
from board_labeler import dctInv

scaler = StandardScaler()

stringLen = len(filesDir)
m = re.compile(r"(\/)([a-zA-Z_]+)(\/)")

X = [cv2.imread(x) for x in paths]
y = [m.findall(x)[0][1] for x in paths]

y = [[dictClasses[x]] for x in y]

print(f"{len(X)} {len(y)}")

8000 8000


In [3]:
from sklearn.model_selection import train_test_split
import numpy as np

X = np.asanyarray(X)
y = np.asanyarray(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
import shutil

train_path = 'train_data/'
test_path = 'test_data/'

if os.path.exists(train_path) and os.path.isdir(train_path):
    shutil.rmtree(train_path)
os.mkdir(train_path)

if os.path.exists(test_path) and os.path.isdir(test_path):
    shutil.rmtree(test_path)
os.mkdir(test_path)

for key, value in dictClasses.items():
    os.mkdir(f"{train_path}[{value}]")
    os.mkdir(f"{test_path}[{value}]")

for i in range(len(X_train)):
    cv2.imwrite(f"{train_path}{y_train[i]}/{i}.png", X_train[i])
    
for i in range(len(X_test)):
    cv2.imwrite(f"{test_path}{y_test[i]}/{i}.png", X_test[i])

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

imgShape = X_train[0].shape
num_classes = len(dictClasses)
print(num_classes)

inner_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=X_train[0].shape,
    classes=num_classes,
)
inner_model.trainable = False
model = keras.Sequential([
    keras.Input(shape=X_train[0].shape),
    layers.RandomContrast(0.5),
    layers.RandomTranslation(0.3, 0.3),
    layers.RandomZoom(0.3, 0.3, fill_mode="constant", interpolation="bilinear"),
    layers.RandomRotation(0.5, fill_mode="constant", interpolation="bilinear"),
    inner_model,
    layers.Flatten(),
    layers.Dense(200, activation='relu'),
    layers.Dense(100, activation='relu'),
    #Dense(50, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax'),
])

model.compile(
    optimizer='adam',   
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.fit(
    X_train,
    keras.utils.to_categorical(y_train),
    epochs=20,
    batch_size=128,
)

13
Epoch 1/20
42/42 [==============================] - 20s 378ms/step - loss: 1.2985 - accuracy: 0.7254
Epoch 2/20
42/42 [==============================] - 15s 363ms/step - loss: 0.5417 - accuracy: 0.8291
Epoch 3/20
42/42 [==============================] - 15s 361ms/step - loss: 0.4531 - accuracy: 0.8616
Epoch 4/20
42/42 [==============================] - 14s 330ms/step - loss: 0.4135 - accuracy: 0.8634
Epoch 5/20
42/42 [==============================] - 14s 327ms/step - loss: 0.3921 - accuracy: 0.8761
Epoch 6/20
42/42 [==============================] - 14s 345ms/step - loss: 0.3414 - accuracy: 0.8869
Epoch 7/20
42/42 [==============================] - 14s 331ms/step - loss: 0.3371 - accuracy: 0.8966
Epoch 8/20
42/42 [==============================] - 15s 354ms/step - loss: 0.3288 - accuracy: 0.9004
Epoch 9/20
42/42 [==============================] - 17s 405ms/step - loss: 0.2904 - accuracy: 0.9017
Epoch 10/20
42/42 [==============================] - 16s 379ms/step - loss: 0.2877 - acc

In [ ]:
score = model.evaluate(X_test, keras.utils.to_categorical(y_test), verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.37414637207984924
Test accuracy: 0.8696969747543335


In [ ]:
model.save("models/secondmodel")

INFO:tensorflow:Assets written to: models/secondmodel\assets


C:\Users\leon.muller\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
